<p><a name="pan"></a></p>

# **Operaciones sobre los datos en Pandas**

Una de las piezas esenciales de NumPy es la capacidad de realizar operaciones vectorizadas, tanto con aritmética básica (suma, resta, multiplicación, etc.) como con operaciones más sofisticadas (funciones trigonométricas, funciones exponenciales y logarítmicas, etc.). Pandas hereda gran parte de esta funcionalidad de NumPy.

Sin embargo, Pandas incluye un par de elementos útiles: para operaciones unarias como funciones de negación y trigonométricas, estas ufuncs conservarán etiquetas de índice y columna en la salida, y para operaciones binarias como suma y multiplicación, Pandas alineará automáticamente los índices. Debido a que Pandas está diseñado para funcionar con NumPy, cualquier ufunc de NumPy funcionará en objetos Pandas Series y DataFrame.

Comencemos por definir una serie y un DataFrame simples con los cuales demostrar esto:

In [ ]:
import numpy as np
import pandas as pd

**Preservando el índice**

Si aplicamos un ufunc de NumPy en cualquiera de estos objetos, el resultado será otro objeto de Pandas con los índices preservados:


**Alineación del índice**


**Series**

Para operaciones binarias con dos objetos (Series o DataFrames), Pandas alineará los índices en el proceso de realizar la operación.


In [ ]:
d_poblacion = {"Brasil":210147125, "Colombia": 50372424, "Argentina":44938712}



In [ ]:
d_area = {"Brasil":8514877, "Colombia":1141748, "Perú": 2792600}



Veamos qué sucede cuando dividimos estas series para calcular la densidad de población:

El arreglo resultante contiene la unión de los índices de los dos arreglos de entrada, que podrían determinarse utilizando la aritmética de conjuntos estándar de Python sobre estos índices:


In [ ]:
# union de los indices


Cualquier elemento para el que una serie u otra no tenga una entrada se marca con `NaN` ("Not a Number"), que es la forma en que Pandas marca los datos faltantes.

Esta coincidencia de índices se implementa de esta manera para cualquiera de las expresiones aritméticas integradas de Python; los valores faltantes se rellenan con `NaN` de forma predeterminada

Si no queremos incluir `NaN` en la salida, podemos utilizar la ufunc directamente. Por ejemplo, llamar a `A.add(B)` es equivalente a llamar a `A + B`, pero permite la especificación explícita opcional del valor de relleno para cualquier elemento en `A` o `B` que pueda faltar, mediante al kwarg `fill_value`:

Note que este kwarg reemplaza `0` en los valores faltantes en las series, no directamente los `NaN` en la salida de la operación de suma.

**DataFrames**

Cuando se realizan operaciones en DataFrames se produce un tipo similar de alineación tanto para las columnas como para los índices:


Observe que los índices están alineados correctamente independientemente de su orden en los dos objetos, y los índices del resultado están ordenados. Como con las series, podemos usar la ufunc asociada y utilizar el kwarg`fill_value`


<p><a name="ope"></a></p>

# **Operaciones entre DataFrame y Series**

Al realizar operaciones entre un DataFrame y una Serie, la alineación del índice y la columna se mantiene de manera similar. Las operaciones entre un DataFrame y una Serie son similares a las operaciones entre un arreglo de NumPy bidimensional y uno unidimensional.

La resta entre un un arreglo bidimensional y una de sus filas se aplica por filas. En Pandas, la convención opera de manera similar:

Si, en cambio, deseamos operar sobre las columnas, podemos especificar el eje explícitamente con el kwarg `axis`. Note que en este caso los índices de la fila deben coincidir con los índices del DataFrame, a diferencia del caso anterior en el que los índices de la Series coincidia con las columnas del DataFrame:


Estudiemos este tipo de operaciones utilizando un conjunto de datos real. En este caso utilizaremos el conjunto de datos `auto.csv` que contiene muestras de vehículos con una serie de características:

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/tomasate/Diplomado_ML/main/datasets/data/nivel_1/Data/auto.csv")
df.head()

,Unnamed: 0,symboling,normalized-losses,make,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,length,width,height,curb-weight,engine-type,num-of-cylinders,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price,city-L/100km,horsepower-binned,diesel,gas
0,0,3,122,alfa-romero,std,two,convertible,rwd,front,88.6,0.811148,0.890278,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111.0,5000.0,21,27,13495.0,11.190476,Medium,0,1
1,1,3,122,alfa-romero,std,two,convertible,rwd,front,88.6,0.811148,0.890278,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111.0,5000.0,21,27,16500.0,11.190476,Medium,0,1
2,2,1,122,alfa-romero,std,two,hatchback,rwd,front,94.5,0.822681,0.909722,52.4,2823,ohcv,six,152,mpfi,2.68,3.47,9.0,154.0,5000.0,19,26,16500.0,12.368421,Medium,0,1
3,3,2,164,audi,std,four,sedan,fwd,front,99.8,0.848630,0.919444,54.3,2337,ohc,four,109,mpfi,3.19,3.40,10.0,102.0,5500.0,24,30,13950.0,9.791667,Medium,0,1
4,4,2,164,audi,std,four,sedan,4wd,front,99.4,0.848630,0.922222,54.3,2824,ohc,five,136,mpfi,3.19,3.40,8.0,115.0,5500.0,18,22,17450.0,13.055556,Medium,0,1


Para comenzar, note que la columna `Unnamed: 0` corresponde a la información del índice, por lo que es una columna que no debemos incluir en el conjunto de datos.

Con el método `drop` podemos eliminar una columna, una fila, o una combinación de ambas (mirar documentación del método). El método no es *in place* por lo que debemos utilizar el kwarg `inplace` para hacer el cambio efectivo sobre el DataFrame

Otra opción es utilizar el kwarg `index_col` de la función `read_csv` con el cual definimos cuál será el índice del DataFrame:

Una vez eliminada esta columna, realicemos algunas operaciones útiles.

Por ejemplo, si quisieramos construir una nueva columna que nos muestre la relación *horsepower* / *price* podemos utilizar simplemente una operación vectorizada utilizando el operador asociado a la ufunc:

Para ordenar esta columna podemos utilizar el método `sort_values`. Con el kwarg `ascending` podemos controlar si el ordenamiento es ascendente o no:

Vemos que la observación con índice 163 contiene la mejor relación *horsepower* / *price*. Accedamos a esta observación, mirando por ejemplo la columna asociada a la constructora *make*:

En muchas ocaciones vamos a querer acceder a esta información, sin tener la necesidad de realizar una visualización para obtener el índice asociado a la observación con el valor mayor. Para este fin podemos utilizar el método `idxmax`:

Otra operación común que se aplica sobre los datos numéricos es el da la normalización de una variable (observe que el conjunto de datos ya contiene este tipo de datos numéricos). Podemos visualizar rápidamente una variable utilizando una operación vectorizada similar a la anterior. Hagamos esto para la variable `height`:

Otro tipo de operación común es la de asignar un valor numérico a una variable categórica de tipo `str`. Por ejemplo, si quisieramos modificar la variable `num-of-doors`



Pasando de tener los valores "four" y "two" a 4 y 2, respectivamente, podemos utilizar la función nativa de Python `map(f, a)`. Esta toma una lista o arreglo `a` y le aplica alguna función `f` a cada uno de los elementos de `a` (similar a lo que hacen las ufuncs de NumPy).

Veamos un ejemplo: creemos el arreglo `[1, 2, 3, 4]` y apliquemos `map` pasando como argumento una función que devuelva el cuadrado de los elementos:

Note que `map` no devuelve una lista sino un generador, por lo que utilizamos el constructor `list` para obtener una lista.

También podemos utilizar esta función para definir un mapeo de una variable a otra, mediante el uso de un diccionario, donde las claves representarán el valor que contiene el arreglo, y las claves los nuevos valores que les queremos dar.

En el caso de una serie, podemos utilizar `map` como un método del objeto



Modifiquemos la variable en nuestro conjunto de datos:

Más adelante veremos formas alternativas para realizar este tipo de operaciones de manera que las podamos incluir en un proceso más general.

<p><a name="agg"></a></p>

# **Agregaciones y agrupamiento**

Una parte esencial del análisis de grandes cantidades de datos es el cálculo de agregaciones como la suma, media, mediana, etc; que nos proporcionan un resumen estadístico de los datos. En esta sección, exploraremos agregaciones en Pandas.

En la sesión anterior vimos cómo aplicar funciones de agregación simples sobre los objetos de pandas, similares a las que hemos visto en los arreglos de NumPy. Veamos ahora algunas funcionalidades más complejas:



**Aggregate (agg)**:

Este método permite tener mucha más flexibilidad en las operaciones de agregación. Puede tomar una cadena de caracteres, una función o una lista de las mismas y calcular todos las agregaciones a la vez. Veamos algunos ejemplos:

In [ ]:
# aplicando agg sobre una serie numerica
# para obtener la media, el maximo
# y el minimo


In [ ]:
# aplicando agg sobre una serie categorica
# para obtener el numero de elemntos
# y el numero de valores unicos


También lo podemos aplicar sobre el dataframe completo y utilizar un diccionario para mapear qué funciones (valores) queremos aplicar a una columna particular (claves)

**Apply**

Este método nos permite aplicar una función arbitraria. La función debe tomar un DataFrame y devolver un objeto de Pandas o un número.

Por ejemplo, supongamos que queremos pasar de tener la variable continua *price* a una variable categórica que tome los valores *inf* y *sup* de acuerdo a un criterio particular. Podemos utilizar la media como criterio de separación:

In [ ]:
# definicion de la funcion a pasar
# al metodo apply
def categoria(df):
  if df.price >=  media:
    return 'sup'
  else:
    return 'inf'




Pasemos esta función al método indicando que queremos realizar la operación por filas:

Modifiquemos la variable en nuestro conjunto de datos:

Note que en la función que se construya para pasar al método `apply`, se pueden evaluar condiciones sobre múltiples columnas para realizar una operación, aunque esta sólo actúe sobre una de ellas. Más adelante veremos ejemplos donde esto pueda ser útil.

Este tipo de operación, aunque nos sirvió para ilustrar el método, se puede realizar fácilmente con la función `cut` de Pandas, la cual nos permite segmentar una variable (*price* en este caso) en ciertos *bins*, a los cuales podemos asociar de forma opcional una etiqueta mediante el kwarg `labels`.

Por ejemplo, realicemos la conversión anterior pero ahora incluyendo un tercer valor adicional `med`:

In [ ]:
p

Modifiquemos la variable en nuestro conjunto de datos:

<p><a name="gro"></a></p>

## **GroupBy: dividir, aplicar y combinar**

Las agregaciones simples pueden darnos una idea del conjunto de datos, pero a menudo preferimos agregar condicionalmente sobre alguna etiqueta o índice: esto se implementa en la llamada operación *groupby*

El objeto GroupBy es una abstracción muy flexible. En muchos sentidos, podemos tratarlo como si fuera una colección de DataFrames, para los cuales se realizan una serie de operaciones complejas "por debajo".

En la siguiente figura se ilustra un ejemplo canónico de esta operación dividir-aplicar-combinar, donde "aplicar" hace referencia a la función de agregación *apply*:

![groupby](https://lewtun.github.io/dslectures/images/split-apply-combine.png)

* El paso de división implica dividir y agrupar un DataFrame según el valor de la clave especificada.
* El paso de aplicación implica calcular alguna función, generalmente una agregación, dentro de los grupos individuales.
* El paso de combinación fusiona los resultados de estas operaciones en un objeto (Serie o DataFrame) como salida.

Si bien esto ciertamente se podría hacer manualmente usando alguna combinación de las funcionalidades de enmascaramiento, agregación y fusión descritos anteriormente, lo importante es que las divisiones intermedias no necesitan ser instanciadas explícitamente. El poder de GroupBy es que abstrae estos pasos: el usuario no necesita pensar en cómo se realiza el cálculo "por debajo", sino que piensa en la operación como un todo.

Veamos un ejemplo concreto con el conjunto de datos que hemos cargado



In [ ]:
df.head()

,symboling,normalized-losses,make,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,length,width,height,curb-weight,engine-type,num-of-cylinders,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price,city-L/100km,horsepower-binned,diesel,gas,ratio,categoría_precio
0,3,122,alfa-romero,std,2,convertible,rwd,front,88.6,0.811148,0.890278,0.816054,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111.0,5000.0,21,27,13495.0,11.190476,Medium,0,1,0.008225,inf
1,3,122,alfa-romero,std,2,convertible,rwd,front,88.6,0.811148,0.890278,0.816054,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111.0,5000.0,21,27,16500.0,11.190476,Medium,0,1,0.006727,inf
2,1,122,alfa-romero,std,2,hatchback,rwd,front,94.5,0.822681,0.909722,0.876254,2823,ohcv,six,152,mpfi,2.68,3.47,9.0,154.0,5000.0,19,26,16500.0,12.368421,Medium,0,1,0.009333,inf
3,2,164,audi,std,4,sedan,fwd,front,99.8,0.848630,0.919444,0.908027,2337,ohc,four,109,mpfi,3.19,3.40,10.0,102.0,5500.0,24,30,13950.0,9.791667,Medium,0,1,0.007312,inf
4,2,164,audi,std,4,sedan,4wd,front,99.4,0.848630,0.922222,0.908027,2824,ohc,five,136,mpfi,3.19,3.40,8.0,115.0,5500.0,18,22,17450.0,13.055556,Medium,0,1,0.006590,inf


Elijamos como clave de agrupación la variable `make`:

Note que lo que se devuelve no es un conjunto de DataFrames, sino un objeto `DataFrameGroupBy`. Para producir un resultado, podemos aplicar un agregado a este objeto, que realizará los pasos apropiados de aplicación / combinación para producir el resultado deseado. Por ejemplo apliquemos la función de agregación `count`:


Note que obtenemos como salida cuántos elementos, por constructora, aparecen en cada columna. Obviamente todas las columnas tienen el mismo valor, por lo que podemos seleccionar alguna columna que queramos visualizar

Alternativamente podemos utilizar el método `size`:

Si ordenamos esta salida en forma descendente obtendremos la misma salida de la aplicación del método `value_counts` sobre la serie `df.make`:

En este caso obviamente la mejor opción es utilizar el método `value_counts`, lo importante es ver la libertad que nos proporciona el `groupby` para agrupar y realizar operaciones sobre los datos.

Veamos algunos otros ejemplos:

In [ ]:
# agrupar por constructora y obtener el minimo y maximo precio
# ordenando la salida por la columna "min"


Incluyamos ahora otra función que nos mida la diferencia entre el precio máximo y el mínimo, y ordenemos de forma descendente respecto a esta columna:

In [ ]:
def difference(s):
  return s.max() - s.min()



Ahora, *groupby* nos permite agrupar por más de una clave. Agrupemos ahora por constructura y gasto de combustible en ciudad "city-mpg" *(miles per gallon)* para obtener los valores mínimos, máximos y su respectiva diferencia para los precios:

Note que en este caso hemos obtenido un DataFrame que contiene no un índice sino dos. Más adelante estudiaremos este tipo de estructuras.

**Ejercicio 1**

Construya un DataFrame a partir del siguiente diccionario

`exam_data = {'name': ['Anastasia', 'Dima', 'Katherine', 'James', 'Emily', 'Michael', 'Matthew', 'Laura', 'Kevin', 'Jonas'], 'score': [12.5, 9, 16.5, 10, 9, 20, 14.5, 12, 8, 19], 'attempts': [1, 3, 2, 3, 2, 3, 1, 1, 2, 1], 'qualify': ['yes', 'no', 'yes', 'no', 'no', 'yes', 'yes', 'no', 'no', 'yes']}`

con las siguientes etiquetas para los índices

`labels = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j']`

* Seleccione solo las columnas *name* y *score*.
  
* Filtre las filas donde el número de intentos (attemps) sea mayor que 2.

* Filtre las filas donde el *score* se encuentre entre 15 y 20 (incluidos)

* Cambie el *score* en la fila 'd' a 11.5

* Agregue una nueva fila con etiqueta 'k' al dataframe con un valor que usted desee para cada columna

* Borre la fila 'k' y entregue nuevamente el DataFrame original.

* Ordene el DataFrame por nombre en orden ascendente.

* Reemplace los valores de la columna *qualify* que contiene los valores *yes* y *no* por los valores booleanos True y False respectivamente.

Importe el dataset `athletes` del repositorio del curso y realice las siguientes tareas:

- Importarlo como nombre athletes
- Filtre el dataframe tal que solo se quede con los atletas que hayan ganado al menos una medalla.
- Calcule la masa muscular por fila con una función como

$$I_{m.m.} = \frac{Weight}{Height^2}$$
- Agrupe los datos por calculando la media por deporte para cada una del resto de características. ¿Cuál es el deporte en el que los jugadores tienen mayor peso promedio?

- ¿Cuál es el pais con más participación femenina?

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/tomasate/Diplomado_ML/main/datasets/data/nivel_1/Data/athletes.csv')